In [3]:
import pandas as pd

from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit

In [96]:
df_nutrients = pd.read_csv("rdi.csv")

In [97]:
df_nutrients

,Energy (kcal/d),Protein (g/d),Carbohydrate (g/d),Fat (g/d),Calcium (mg/d),Chloride (mg/d),Copper (mg/d),Iron (mg/d),Iodine (µg/d),Magnesium (mg/d),Manganese (mg/d),Phosphorus (mg/d),Potassium (mg/d),Selenium (µg/d),Sodium (mg/d),Zinc (mg/d),Vitamin D (µg/d),Vitamin E (mg/d),Vitamin K1 (µg/d),Vitamin C (mg/d),Vitamin B1 or Thiamin (mg/d),Vitamin B2 or Riboflavin (mg/d),Vitamin B3 or Niacin (mg/d),Vitamin B5 or Pantothenic acid (mg/d),Vitamin B6 (mg/d),Vitamin B9 or Folate (µg/d),Vitamin B12 (µg/d),Vitamin A (µg/d)
0,2500.0,66.4,281.25,97.222222,1000.0,3100.0,1.6,11.0,150.0,350.0,3.0,550.0,3500.0,70.0,2000.0,10.0,15.0,13.0,70.0,110.0,1.0467,1.6,16.7472,5.0,1.7,330.0,4.0,750.0


In [98]:
nutrients = [(key,value) for key,value in df_nutrients.to_dict("records")[0].items()]
nutrients

[('Energy (kcal/d)', 2500.0),
 ('Protein (g/d)', 66.39999999999999),
 ('Carbohydrate (g/d)', 281.25),
 ('Fat (g/d)', 97.22222222222224),
 ('Calcium (mg/d)', 1000.0),
 ('Chloride (mg/d)', 3100.0),
 ('Copper (mg/d)', 1.6),
 ('Iron (mg/d)', 11.0),
 ('Iodine (µg/d)', 150.0),
 ('Magnesium (mg/d)', 350.0),
 ('Manganese (mg/d)', 3.0),
 ('Phosphorus (mg/d)', 550.0),
 ('Potassium (mg/d)', 3500.0),
 ('Selenium (µg/d)', 70.0),
 ('Sodium (mg/d)', 2000.0),
 ('Zinc (mg/d)', 10.0),
 ('Vitamin D (µg/d)', 15.0),
 ('Vitamin E (mg/d)', 13.0),
 ('Vitamin K1 (µg/d)', 70.0),
 ('Vitamin C (mg/d)', 110.0),
 ('Vitamin B1 or Thiamin (mg/d)', 1.0467000000000002),
 ('Vitamin B2 or Riboflavin (mg/d)', 1.6),
 ('Vitamin B3 or Niacin (mg/d)', 16.747200000000003),
 ('Vitamin B5 or Pantothenic acid (mg/d)', 5.0),
 ('Vitamin B6 (mg/d)', 1.7),
 ('Vitamin B9 or Folate (µg/d)', 330.0),
 ('Vitamin B12 (µg/d)', 4.0),
 ('Vitamin A (µg/d)', 750.0)]

In [118]:
df_foods=pd.read_csv("ciqual_2020.csv")

In [119]:
# https://stackoverflow.com/questions/18172851/deleting-dataframe-row-in-pandas-based-on-column-value
def filter_rows_by_values(df, col, values):
    return df[~df[col].isin(values)]

In [243]:
# remove certain foods

remove_foods = ["Acerola. pulp. raw. sampled in the island of La Martiniqu", 
                "Egg. powd", "Milk. powder. semi-skimmed", 
                "Decaffeinated coffee. powder. instan", 
                "Royal jelly", 
                "Cocoa powder for baby beverag", 
                "Egg white. powd", 
                "Milk. powder. skimmed", 
                "Instant cereal (powder to be reconstituted) for baby from 4/6 month",
                "Milk. powder. whol",
                "Instant cereal (powder to be reconstituted) for baby from 6 month",
                "Egg yolk. powd", 
                "Gelatine. dried", 
                "Baby milk. first age. powd", 
                "Soya flou", 
                "Sea belt (Saccharina latissima). dried or dehydrated", 
                "Veal stock for sauce and cooking. dehydrated", 
                "Broth. stock or bouillon. meat and vegetables. with fat. dehydrated", 
                "Broth. stock or bouillon. meat and vegetables. defatted. dehydrated"]

df_foods_filtered = filter_rows_by_values(df_foods, "Name", remove_foods)

In [244]:
commodities = list(df_foods_filtered["Name"])

In [245]:
data = df_foods_filtered.drop("Name", axis=1).values.tolist()

In [246]:
solver = pywraplp.Solver.CreateSolver('GLOP')

In [247]:
# Declare an array to hold our variables. 
foods = [solver.NumVar(0.0, solver.infinity(), item) for item in commodities]

print('Number of variables =', solver.NumVariables())

Number of variables = 3167


In [248]:
# Create the constraints, one per nutrient. (data = nutrients_per_100_gramm)
# gurobipy can express a lists or arrays of constraints with a nicer DSL 
# instead of the many loops necessary with OR-Tools
constraints = []
for i, nutrient in enumerate(nutrients):
    constraints.append(solver.Constraint(nutrient[1], solver.infinity()))
    for j, item in enumerate(data):
        constraints[i].SetCoefficient(foods[j], item[i])

print('Number of constraints =', solver.NumConstraints())

Number of constraints = 28


In [249]:
# Objective function: Minimize the sum of (price-normalized) foods.
objective = solver.Objective()
for i, food in enumerate(foods):
    objective.SetCoefficient(food, 1.0)
objective.SetMinimization()

In [250]:
status = solver.Solve()

# Check that the problem has an optimal solution.
if status != solver.OPTIMAL:
    print('The problem does not have an optimal solution!')
    if status == solver.FEASIBLE:
        print('A potentially suboptimal solution was found.')
    else:
        print('The solver could not solve the problem.')
        exit(1)

In [251]:
# Display the amounts (in dollars) to purchase of each food.
nutrients_result = [0] * len(nutrients)
print('\nDaily Foods:')
for i, food in enumerate(foods):
    if food.solution_value() > 0.0:
        print('{}: {} gr'.format(commodities[i], food.solution_value()*100))
        for j, _ in enumerate(nutrients):
            nutrients_result[j] += data[i][j] * food.solution_value()
print('\nOptimal daily weight: {:.4f} gr'.format(objective.Value()*100))


Daily Foods:
Potato crisps. "à l'ancienne" (old-fashioned style): 130.43290777632097 gr
Cashew nut. dry-grilled. unsalted: 93.36607916109652 gr
Soybean. whole grain: 54.368128906788215 gr
Rusk: 53.62822255490002 gr
Breakfast cereals. diet. plain or with honey. fortified with vitamins and chemical elemen: 135.7365431149498 gr
Breakfast cereals. diet. with fruits. fortified with vitamins and chemical elemen: 15.676562755538637 gr
Breakfast cereals. puffed/popped corn. with honey (not fortified with vitamins and chemical elements): 26.635398436050906 gr
Cod liver oil: 5.695937392071632 gr
Salt. white (sea. igneous or rock). no enrichmen: 0.2555727809988329 gr
Parsley. dried: 0.5788254642337439 gr
Kombu or Japanese kelp (Laminaria japonica). dried or dehydrated: 0.031705083003349635 gr

Optimal daily weight: 516.4059 gr


In [252]:
print('\nNutrients per day:')
for i, nutrient in enumerate(nutrients):
    print('{}: {:.2f} (min {})'.format(nutrient[0], nutrients_result[i],
                                       nutrient[1]))


Nutrients per day:
Energy (kcal/d): 2500.00 (min 2500.0)
Protein (g/d): 66.40 (min 66.39999999999999)
Carbohydrate (g/d): 281.25 (min 281.25)
Fat (g/d): 117.26 (min 97.22222222222224)
Calcium (mg/d): 1000.00 (min 1000.0)
Chloride (mg/d): 3100.00 (min 3100.0)
Copper (mg/d): 3.35 (min 1.6)
Iron (mg/d): 33.87 (min 11.0)
Iodine (µg/d): 150.00 (min 150.0)
Magnesium (mg/d): 565.37 (min 350.0)
Manganese (mg/d): 3.90 (min 3.0)
Phosphorus (mg/d): 1317.11 (min 550.0)
Potassium (mg/d): 3500.00 (min 3500.0)
Selenium (µg/d): 70.00 (min 70.0)
Sodium (mg/d): 2118.41 (min 2000.0)
Zinc (mg/d): 10.48 (min 10.0)
Vitamin D (µg/d): 15.00 (min 15.0)
Vitamin E (mg/d): 39.80 (min 13.0)
Vitamin K1 (µg/d): 70.00 (min 70.0)
Vitamin C (mg/d): 144.08 (min 110.0)
Vitamin B1 or Thiamin (mg/d): 3.63 (min 1.0467000000000002)
Vitamin B2 or Riboflavin (mg/d): 3.47 (min 1.6)
Vitamin B3 or Niacin (mg/d): 40.73 (min 16.747200000000003)
Vitamin B5 or Pantothenic acid (mg/d): 14.74 (min 5.0)
Vitamin B6 (mg/d): 4.03 (min 1.7

In [253]:
nutrient_per_food = {}

for i, food in enumerate(foods):
    if food.solution_value() > 0.0:      
        for j, nutrient in enumerate(nutrients):
            if food in nutrient_per_food:
                nutrient_per_food[food].append(data[i][j] * food.solution_value())
            else:
                nutrient_per_food[food]=[data[i][j] * food.solution_value()]
                
foods_df = pd.DataFrame.from_dict(nutrient_per_food, orient='index', columns=[n[0] for n in nutrients])

for i, nutrient in enumerate(nutrients):
    foods_df[nutrient[0]]=(foods_df[nutrient[0]]/nutrients_result[i]*100).round(2)

display(foods_df)     

,Energy (kcal/d),Protein (g/d),Carbohydrate (g/d),Fat (g/d),Calcium (mg/d),Chloride (mg/d),Copper (mg/d),Iron (mg/d),Iodine (µg/d),Magnesium (mg/d),Manganese (mg/d),Phosphorus (mg/d),Potassium (mg/d),Selenium (µg/d),Sodium (mg/d),Zinc (mg/d),Vitamin D (µg/d),Vitamin E (mg/d),Vitamin K1 (µg/d),Vitamin C (mg/d),Vitamin B1 or Thiamin (mg/d),Vitamin B2 or Riboflavin (mg/d),Vitamin B3 or Niacin (mg/d),Vitamin B5 or Pantothenic acid (mg/d),Vitamin B6 (mg/d),Vitamin B9 or Folate (µg/d),Vitamin B12 (µg/d),Vitamin A (µg/d)
"Potato crisps. ""à l'ancienne"" (old-fashioned style)",29.74,11.04,23.10,41.82,2.74,45.02,7.39,3.77,17.39,12.69,8.36,11.88,40.99,37.27,43.22,7.47,2.17,50.14,8.50,4.59,11.14,0.75,15.18,7.34,11.33,5.53,0.00,1.58
Cashew nut. dry-grilled. unsalted,23.04,24.47,7.80,39.02,3.55,0.72,75.18,15.71,0.00,46.24,43.10,37.57,18.14,26.68,0.35,49.91,1.56,1.36,41.75,0.32,9.26,1.32,1.28,6.21,3.94,3.71,0.00,0.08
Soybean. whole grain,9.11,28.25,4.02,8.90,11.96,0.00,14.27,25.20,0.22,24.33,31.51,24.19,27.03,6.83,0.08,15.31,0.00,1.16,36.50,2.26,13.03,13.63,2.16,5.02,5.40,27.71,0.00,0.07
Rusk,8.69,7.92,14.61,2.70,1.56,15.33,2.56,1.74,7.15,2.28,8.53,4.07,2.45,15.32,12.89,4.04,0.89,1.12,1.70,0.00,1.77,0.15,0.92,1.89,1.08,1.86,0.00,1.58
Breakfast cereals. diet. plain or with honey. fortified with vitamins and chemical elemen,20.74,23.30,37.93,1.91,71.94,29.99,0.00,42.48,4.52,11.40,0.00,18.55,8.65,0.00,31.85,19.44,0.00,37.52,0.00,83.75,52.71,67.67,64.99,64.28,63.66,49.14,78.05,0.00
Breakfast cereals. diet. with fruits. fortified with vitamins and chemical elemen,2.37,2.48,4.34,0.23,5.55,0.00,0.00,4.91,2.09,1.62,6.43,2.02,1.06,6.23,3.14,2.24,0.00,3.98,0.00,8.42,5.40,6.87,6.62,6.16,6.50,4.99,7.96,0.00
Breakfast cereals. puffed/popped corn. with honey (not fortified with vitamins and chemical elements),4.19,2.29,8.16,0.53,2.29,3.93,0.48,5.50,3.55,0.99,1.23,1.48,0.84,7.61,3.61,1.22,0.44,0.30,0.30,0.09,6.60,9.21,8.70,9.04,7.93,6.87,13.98,0.46
Cod liver oil,2.05,0.00,0.00,4.86,0.01,0.00,0.01,0.01,15.19,0.00,0.00,0.00,0.00,0.00,0.00,0.03,94.93,4.29,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,95.02
Salt. white (sea. igneous or rock). no enrichmen,0.00,0.00,0.00,0.00,0.00,5.01,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.00,4.72,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Parsley. dried,0.07,0.25,0.03,0.03,0.38,0.00,0.11,0.65,0.00,0.40,0.83,0.22,0.74,0.00,0.12,0.32,0.00,0.13,11.25,0.55,0.10,0.39,0.14,0.07,0.16,0.17,0.00,0.06


In [254]:
foods_df.sum()

Energy (kcal/d)                          100.00
Protein (g/d)                            100.00
Carbohydrate (g/d)                        99.99
Fat (g/d)                                100.00
Calcium (mg/d)                           100.01
Chloride (mg/d)                          100.00
Copper (mg/d)                            100.01
Iron (mg/d)                               99.98
Iodine (µg/d)                             99.99
Magnesium (mg/d)                         100.01
Manganese (mg/d)                         100.00
Phosphorus (mg/d)                        100.00
Potassium (mg/d)                         100.00
Selenium (µg/d)                          100.00
Sodium (mg/d)                            100.02
Zinc (mg/d)                               99.98
Vitamin D (µg/d)                          99.99
Vitamin E (mg/d)                         100.00
Vitamin K1 (µg/d)                        100.00
Vitamin C (mg/d)                          99.98
Vitamin B1 or Thiamin (mg/d)            

In [255]:
nutrient_per_food = {}

for i, food in enumerate(foods):
    if food.solution_value() > 0.0:      
        for j, nutrient in enumerate(nutrients):
            if food in nutrient_per_food:
                nutrient_per_food[food].append(data[i][j] * food.solution_value())
            else:
                nutrient_per_food[food]=[data[i][j] * food.solution_value()]
                
foods_df = pd.DataFrame.from_dict(nutrient_per_food, orient='index', columns=[n[0] for n in nutrients])

for i, nutrient in enumerate(nutrients):
    foods_df[nutrient[0]]=(foods_df[nutrient[0]]).round(2)

# foods_df.loc['total'] = foods_df.iloc[1:, :-1].sum()    
    
display(foods_df)  

,Energy (kcal/d),Protein (g/d),Carbohydrate (g/d),Fat (g/d),Calcium (mg/d),Chloride (mg/d),Copper (mg/d),Iron (mg/d),Iodine (µg/d),Magnesium (mg/d),Manganese (mg/d),Phosphorus (mg/d),Potassium (mg/d),Selenium (µg/d),Sodium (mg/d),Zinc (mg/d),Vitamin D (µg/d),Vitamin E (mg/d),Vitamin K1 (µg/d),Vitamin C (mg/d),Vitamin B1 or Thiamin (mg/d),Vitamin B2 or Riboflavin (mg/d),Vitamin B3 or Niacin (mg/d),Vitamin B5 or Pantothenic acid (mg/d),Vitamin B6 (mg/d),Vitamin B9 or Folate (µg/d),Vitamin B12 (µg/d),Vitamin A (µg/d)
"Potato crisps. ""à l'ancienne"" (old-fashioned style)",743.47,7.33,64.96,49.04,27.39,1395.63,0.25,1.28,26.09,71.74,0.33,156.52,1434.76,26.09,915.64,0.78,0.33,19.96,5.95,6.61,0.40,0.03,6.18,1.08,0.46,35.61,0.00,28.48
Cashew nut. dry-grilled. unsalted,576.07,16.25,21.94,45.75,35.48,22.31,2.52,5.32,0.00,261.43,1.68,494.84,634.89,18.67,7.47,5.23,0.23,0.54,29.22,0.47,0.34,0.05,0.52,0.91,0.16,23.90,0.00,1.45
Soybean. whole grain,227.80,18.76,11.31,10.44,119.61,0.00,0.48,8.54,0.33,137.55,1.23,318.60,946.01,4.78,1.63,1.60,0.00,0.46,25.55,3.26,0.47,0.47,0.88,0.74,0.22,178.33,0.00,1.18
Rusk,217.19,5.26,41.08,3.16,15.55,475.15,0.09,0.59,10.73,12.87,0.33,53.63,85.81,10.73,272.97,0.42,0.13,0.45,1.19,0.00,0.06,0.01,0.38,0.28,0.04,11.96,0.00,28.42
Breakfast cereals. diet. plain or with honey. fortified with vitamins and chemical elemen,518.51,15.47,106.69,2.24,719.40,929.80,0.00,14.39,6.79,64.47,0.00,244.33,302.69,0.00,674.61,2.04,0.00,14.93,0.00,120.67,1.91,2.35,26.47,9.47,2.57,316.27,3.12,0.00
Breakfast cereals. diet. with fruits. fortified with vitamins and chemical elemen,59.26,1.65,12.21,0.27,55.50,0.00,0.00,1.66,3.14,9.14,0.25,26.65,37.15,4.36,66.47,0.24,0.00,1.58,0.00,12.13,0.20,0.24,2.70,0.91,0.26,32.14,0.32,0.00
Breakfast cereals. puffed/popped corn. with honey (not fortified with vitamins and chemical elements),104.68,1.52,22.96,0.63,22.91,121.72,0.02,1.86,5.33,5.59,0.05,19.44,29.30,5.33,76.44,0.13,0.07,0.12,0.21,0.13,0.24,0.32,3.54,1.33,0.32,44.21,0.56,8.22
Cod liver oil,51.26,0.00,0.00,5.70,0.07,0.00,0.00,0.00,22.78,0.00,0.00,0.00,0.00,0.00,0.00,0.00,14.24,1.71,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1708.78
Salt. white (sea. igneous or rock). no enrichmen,0.00,0.00,0.00,0.00,0.03,155.39,0.00,0.00,0.00,0.01,0.00,0.02,0.04,0.00,99.93,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Parsley. dried,1.68,0.17,0.10,0.03,3.81,0.00,0.00,0.22,0.00,2.23,0.03,2.85,25.99,0.00,2.44,0.03,0.00,0.05,7.87,0.79,0.00,0.01,0.06,0.01,0.01,1.08,0.00,1.11


In [256]:
foods_df.sum()

Energy (kcal/d)                          2499.99
Protein (g/d)                              66.41
Carbohydrate (g/d)                        281.26
Fat (g/d)                                 117.26
Calcium (mg/d)                           1000.00
Chloride (mg/d)                          3100.00
Copper (mg/d)                               3.36
Iron (mg/d)                                33.86
Iodine (µg/d)                             150.01
Magnesium (mg/d)                          565.36
Manganese (mg/d)                            3.90
Phosphorus (mg/d)                        1317.12
Potassium (mg/d)                         3500.00
Selenium (µg/d)                            70.00
Sodium (mg/d)                            2118.41
Zinc (mg/d)                                10.47
Vitamin D (µg/d)                           15.00
Vitamin E (mg/d)                           39.80
Vitamin K1 (µg/d)                          69.99
Vitamin C (mg/d)                          144.06
Vitamin B1 or Thiami

# Ideas
- Protein probably a bit too low
- Some constraints are missing:
  - Omega3
  - Fiber
  - Water